__Run basic contamination screening and fastqc. perform mapping and alignment, following this tutorial https://www.biostars.org/p/41951/__

In [ ]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/2022-02-07-fastq_screen/


In [ ]:
#make a list of read filenames to use for next loops
!ls *R2.fastq.gz > filenames
!sed -i'' -e 's/R2.fastq.gz//g' filenames #remove the extension

In [38]:
#there are some samples fastqc failed for both reads per base seuqence content, use samtools/bamstats to look at how those aligned versus samples that didn't fail
%cd ~/ClownfishGWAS/data/APPC_sequencing/DNA
!samtools stats Pop01_APPC1921.F.bam > troubleshoot_reads/Pop01_APPC1921.F.stats
!samtools stats Pop07_APPC2603.F.bam > troubleshoot_reads/Pop07_APPC2603.F.stats


/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA


In [ ]:
#screen for contamination and filter out tagged reads simultaneously, filter 1 means keep only reads that mapped exclusively to the reference for A. percula. since this is already done for the read 1 files, just need to do it for the read 2 files that I cut the barcode from  
!for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/*.cut.fastq.gz; do fastq_screen --tag --filter 1 --threads 32 --outdir /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/ $i; done

In [ ]:
#run fastqc 
!for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/*tagged_filter.fastq.gz; do fastqc -t 6 *tagged_filter.fastq.gz  $i; done

In [ ]:
#visualize sequencing quality for all samples in trimmed_reads directory. check that all barcode sequences are removed
!multiqc .

In [ ]:
!zcat CAP1_APPC_16.11.R1.fastq.gz | head
!zcat CAP1_APPC_16.11.R2.fastq.gz | head



In [45]:
#okay, from the fastqc html, looks like the read 2 file needed trimming to about 15bp
#get mapping stats for sample
%cd ~/ClownfishGWAS/data/APPC_sequencing/DNA/

!samtools stats  ~/ClownfishGWAS/data/APPC_sequencing/DNA/Pop01_APPC1611.F.bam > troubleshoot_reads/Pop01_APPC1611.F.stats

#trim with cutadapt, and then remap to the reference
#get the mapping stats again

/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA
[W::sam_read1] Parse error at line 368
Failure while decoding file


In [ ]:
#trim with cutadapt to remove 17 bp 5', and then remap to the reference
cutadapt -U 17 -o  /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/troubleshoot_reads/Pop01_APPC1611.R1.cut.fastq.gz -p /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/troubleshoot_reads/Pop01_APPC1611.R2.cut.fastq.gz  /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/Pop01_APPC1611.R1.fq.gz  /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/Pop01_APPC1611.R2.fq.gz

In [58]:
#how did it go?
!samtools stats  /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/Pop16_APPC4215-RG.bam > trimmed_reads/mapping/Pop16_APPC4215-RG.stats
!samtools stats  /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/Pop01_APPC2907.F.bam > troubleshoot_reads/Pop01_APPC2907.F.stats
#woohoo! almost 3.5x more mapped reads and 95% of reads properly paired
113175058/33542679

__March 21, 2022 remap with ensembl genome__

In [1]:
#03/22/2021 remap to update ensembl genome
!gunzip /data/katrinac/APPC_sequencing/Genome/ensembl/Amphiprion_percula.Nemo_v1.dna.toplevel.fa.gz
!bwa index /data/katrinac/APPC_sequencing/Genome/ensembl/Amphiprion_percula.Nemo_v1.dna.toplevel.fa

[bwa_index] Pack FASTA... 7.84 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=1817878588, availableWord=139912276
[BWTIncConstructFromPacked] 10 iterations done. 99999996 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 199999996 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 299999996 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 399999996 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 499999996 characters processed.
[BWTIncConstructFromPacked] 60 iterations done. 599999996 characters processed.
[BWTIncConstructFromPacked] 70 iterations done. 699999996 characters processed.
[BWTIncConstructFromPacked] 80 iterations done. 799999996 characters processed.
[BWTIncConstructFromPacked] 90 iterations done. 899999996 characters processed.
[BWTIncConstructFromPacked] 100 iterations done. 999999996 characters processed.
[BWTIncConstructFromPacked] 110 iterations done. 1

In [2]:
%cd /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/
!mkdir ensembl
%cd /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl


/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping
/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl


In [ ]:
!for i in $(cat /data/katrinac/APPC_sequencing/DNA/trimmed_reads/filenames); do bwa mem -t 40 -M -R "@RG\tID:$i\tSM:$i\tPL:Illumina" /data/katrinac/APPC_sequencing/Genome/ensembl/Amphiprion_percula.Nemo_v1.dna.toplevel.fa /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/$i.R1.cut.fastq.gz /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/$i.R2.cut.fastq.gz  2> bwa.$i.log | samtools sort -@ 40 -o /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/$i-RG.bam 2> /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/$i.bam.log; done  


In [7]:
!pwd

/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl


__how did mapping with ensembl vs ncbi genome go?__

In [8]:
#how did it go?
!samtools stats  /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/Pop16_APPC4215-RG.bam > /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/Pop16_APPC4215-RG.stats
#more unmapped reads, but I think that might be the contigs not assigned to chromosomes (PYAM)?

__more unmapped reads, but I think that might be the contigs not assigned to chromosomes (PYAM)?__

In [10]:
%cd /data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl/


/data/katrinac/APPC_sequencing/DNA/trimmed_reads/mapping/ensembl


In [ ]:
#mark duplicates, from Jon's script, remember to cite Jon and GNU parallel
!cat /data/katrinac/APPC_sequencing/DNA/trimmed_reads/filenames | parallel -j 8 "java -Xms4g -jar /local/home/katrinac/miniconda3/envs/commandline/bin/picard.jar MarkDuplicates I={}-RG.bam O={}-RGrd.bam M={}_dup_metrics.txt REMOVE_DUPLICATES=true READ_NAME_REGEX=true  OPTICAL_DUPLICATE_PIXEL_DISTANCE=2500 TAGGING_POLICY=OpticalOnly &> md.{}.log"

Academic tradition requires you to cite works you base your article on.
When using programs that use GNU Parallel to process data for publication
please cite:

  O. Tange (2011): GNU Parallel - The Command-Line Power Tool,
  ;login: The USENIX Magazine, February 2011:42-47.

This helps funding further development; AND IT WON'T COST YOU A CENT.
If you pay 10000 EUR you should feel free to use GNU Parallel without citing.

To silence the citation notice: run 'parallel --citation'.



In [12]:
#check alignments once more
!samtools stats  Pop16_APPC4215-RGrd.bam > Pop16_APPC4215-RGrd.stats

#call genotypes and calculate GL
#use dDocent to call genotypes/snps

In [13]:
!mkdir genotypes
!for i in $(cat /data/katrinac/APPC_sequencing/DNA/trimmed_reads/filenames); do mv $i-RGrd.bam genotypes/$i.F.bam; done #copy the files with duplicates removed by picard matching the ddocent naming convention

__03/23/2023, done remapping with ensembl__

In [91]:
#move reference genome files to the genotype directory
!for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/reference*; do cp /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/reference* /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/; done #copy the files with duplicates removed by picard matching the ddocent naming convention

In [ ]:
#call genotypes with ddocent
%cd ~/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/
#!ln -s /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/Genome/reference.fasta reference.fasta 
!/local/home/katrinac/ClownfishGWAS/bioinformatics_scripts/dDocent_ngs.sh /local/home/katrinac/ClownfishGWAS/bioinformatics_scripts/config_call_genos

/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes
dDocent 2.2.22 

Contact jpuritz@gmail.com with any problems 

 
Checking for required software

All required software is installed!
ls: cannot access '*.F.fq.gz': No such file or directory
ls: cannot access '*.F.fq.gz': No such file or directory

dDocent run started Tue Feb 15 10:38:38 EST 2022 

At this point, all configuration information has been entered and dDocent may take several hours to run.
It is recommended that you move this script to a background operation and disable terminal input and output.
All data and logfiles will still be recorded.
To do this:
Press control and Z simultaneously
Type 'bg' without the quotes and press enter
Type 'disown -h' again without the quotes and press enter

Now sit back, relax, and wait for your analysis to finish.
Using FreeBayes to call SNPs
/usr/bin/bash: line 1: 38027 Killed                  freebayes -b split.$1.bam -t mapped.$1.bed -v raw.$1.vcf 

In [86]:
#check stats
!samtools stats /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/filter.merged.bam > trimmed_reads/mapping/genotypes/filter.merged.stats


/bin/bash: trimmed_reads/mapping/genotypes/filter.merged.stats: No such file or directory


In [ ]:
#try variant calling with bcf tools


!bcftools mpileup -Ou -f /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/reference.fasta /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/filter.merged.bam | bcftools call -mv -Ob -o /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/calls.bcf



Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
[mpileup] 159 samples in 1 input files
Note: The maximum per-sample depth with -d 250 is 1.6x
